<a href="https://colab.research.google.com/github/D3TaLES/databases_demo/blob/main/notebooks/sql_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To view this demonstration, simply click the play button beside each code cell to run the cell. Note that these cells should be run in order. It is also recomended that this notebook is run in Colab.**

# Demonstration of an SQL Database
In this notebook, we demonstrate a SQL database. The SQL database structure is the original data management structure. It contains collections of two-dimensional tables, akin to collections of spreadsheets pages. The table rows are data records, and columns are attributes. Each attribute (column) can contain only a single numerical or text value for each record (row). When a data record has an embedded attribute, an SQL database uses multiple tables. For example, a molecule may contain the attribute `UVVis_Data`; however, `UVVis_Data` contains embedded attributes such as Instrument_Name and Optical_Gap. To accommodate these data, the first table contains the molecule record with its primary key and its regular attributes, while another table contains `UVVis_Data` and its attributes. Each record in `UVVis_Data` connects to the molecules table with a table-joining column. This column contains a molecule primary key. 

**Note**: This schema is not a complete picture of the schema for the data in this notebook; it is only a partial schema. 

<img src='https://raw.githubusercontent.com/D3TaLES/databases_demo/main/notebooks/media/sql_structure.png' width="950" height="300">

After first initializing the database, we generate example data that abide by the SQL schema. We then insert experimental data into the database for various moelcules of varying data types (computational and experimental). Finally, we give examples databaes queries and show how to easily plot queried data.


## Install and Import Needed Code

Here we use `pip install` to install several packages for use in this notebook. We also pull the file processing code and the example data files from our [GitHub repoisitory](https://github.com/D3TaLES/databases_demo/). Then we import the packages so they can be used.  

Note: Colab normally has [pandas](https://pandas.pydata.org/), [numpy](https://numpy.org/), [matplotlib](https://matplotlib.org/), [scipy](https://scipy.org/), and [sqlite3](https://www.sqlite.org/index.html) pre-installed. If you do not have these packages installed, you will need to install them. 

In [ ]:
%%capture
! pip install pymatgen  # Install Pymatgen for Gaussian file parsing 
! pip install pubchempy  # Install PubChem python API for moleucle information
! pip install rdkit-pypi  # Install RdKit for molecule transformations
! pip install qcfractal # Install QCFractal for SQL schema
! pip install sqlalchemy==1.3.* # Install SQLAlchemy for SQL database 

In [ ]:
! rm -r databases_demo/ # Remove database_demo directory if it already exists
! git clone https://github.com/D3TaLES/databases_demo.git # Get Processing code from GitHub

In [ ]:
# Import required packages (many of which you just installed)
import sqlite3
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql.schema import MetaData
from databases_demo.file_parser import *
from databases_demo.schema.sql_schema import *

## 1. Initialize the database 
Here we initalize a [SQLite](https://www.sqlite.org/index.html) SQL database by first connecting to the dabase session engine and starting a session. Then we create a series of database tables (`Molecules`, `DftData`, `Synonyms`, `UvVisData`, and `AbsorbanceData`). Each table corresponds to a table defined in the [SQL schema](https://github.com/D3TaLES/databases_demo/blob/main/schema/sql_schema.py) for this example notebook. 

In [ ]:
# Connect to the database
engine = create_engine('sqlite:///sample.db') 
Session = sessionmaker(bind=engine)
session = Session()

# Create tables
MetaData(bind=engine).create_all(tables=[
                                         Molecules.__table__,
                                         DftData.__table__,
                                         Synonyms.__table__, 
                                         UvVisData.__table__,
                                         AbsorbanceData.__table__
                                         ])

## 2. Load Schema and use it to validate example data

Here we begin the Extract, Transform, and Load (ETL) process to insert data into our newly created database. In this step we complete the extraction and transformation. (In step 3, we will load.) 

<img src='https://raw.githubusercontent.com/D3TaLES/databases_demo/main/notebooks/media/etl.png' width="800" height="300">

We must first generate general molecular information about our molecule, in this case biphenyl. We use the `GenerateMolInfo` module defined in our [processing code](https://github.com/D3TaLES/databases_demo/blob/main/file_parser.py). This module both generates (or extracts) general moleucle information for biphenyl and transforms it to our defined schema. Then we use the same process to generate synnonym data for biphenyl. 

In [ ]:
# EXTRACT/generate basic data for biphenyl and TRANSFORM to schema
data_generation = GenerateMolInfo(smiles="C1=C(c2ccccc2)CCCC1", source='our_lab', names=['biphenyl'])
mol_data = data_generation.data
# Get the generated primary key for biphenyl
bp_id = mol_data.get('mol_id')

mol_data # Show data

In [ ]:
# Get synonym data 
synonym_rawdata = data_generation.synonym_data
# Convert each synonym table row into a validated object
synonym_data = [Synonyms(**data) for data in synonym_rawdata] 

synonym_rawdata[:5] # Show (first 5 pieces of) data

## 3. Insert validated data ino the database
In this section, coplete the final step of the ETL process by loading generated data (from section 2) into the database. This includes adding data to the `Molecules` table and the `Synonyms` table seperately. We also must commit the data after we add it. 

Then, we repeat the entire ETL process for three additional moleucles. And finally, we complete the ETL process for different types of data, namely, computaitonal data and experimental UV-Vis data. Inserting computation/experimental data into the database requires both raw data files and more complex file parsing. Here we use [these raw data files](https://github.com/D3TaLES/databases_demo/tree/main/raw_data), and the parsing is performed by the `ProcessDFT` and `ProcessUvVis` modules defined in our [processing code](https://github.com/D3TaLES/databases_demo/blob/main/file_parser.py). Basic processing demonstrations that reflect the parsing done in these emodules can be found [in this Colab notebook](https://github.com/D3TaLES/databases_demo/blob/main/notebooks/processing_demo.ipynb).

In [ ]:
# LOAD single molecule object to the Molecules table
session.add(Molecules(**mol_data))
session.commit()

# LOAD multiple synonym objects to the Synonym table
session.bulk_save_objects(synonym_data) 
session.commit()


### ETL for different molecules

Here we loop through a dictionary of molecule names and their SMILES strings, and for each, we generate general molecule data and insert the data to the database. 

In [ ]:
# ETL for Benzene, Nitrobenzene, and Anthracene
extra_mols = {'benzene': "C1=CC=CC=C1", 'nitrobenzene': "C1=CC=C(C=C1)[N+](=O)[O-]", 'anthracene': "C1=CC=C2C=C3C=CC=CC3=CC2=C1"}
extra_mol_ids = {}
for name, smiles in extra_mols.items(): 
  # Extract and transform
  mol_data = GenerateMolInfo(smiles, source='our_lab', names=[name]).data
  # Load
  session.add(Molecules(**mol_data))
  session.commit()

  # Record moleucle id
  extra_mol_ids[name] = mol_data.get('mol_id')


### ETL for different types of data

Here we extract and transform computational data from a Gaussian DFT [log file](https://github.com/D3TaLES/databases_demo/tree/main/raw_data/tddft_biphenyl.log), then load the data to the `DftData` database table.

In [ ]:
# EXTRACT and TRANSFORM Gaussian DFT data
gaussian_data = ProcessDFT('databases_demo/raw_data/tddft_biphenyl.log', mol_id=bp_id).data

# LOAD DFT data into database
session.add(DftData(**gaussian_data))
session.commit()

Here we extract and transform experimental UV-Vis data from a UV-Vis [output CSV file](https://github.com/D3TaLES/databases_demo/tree/main/raw_data/uvvis_biphenyl.csv), then load the data to the database tables. A demonstration of the parsing done here can be found [in this Colab notebook](https://github.com/D3TaLES/databases_demo/blob/main/notebooks/processing_demo.ipynb). Note that general information about the UV-Vis experiment goes into the `UvVisData` table, while the absorbance data goes into the `AbsorbanceData` table. 

In [ ]:
# Insert UV-Vis data
uvvis_data = ProcessUvVis('databases_demo/raw_data/uvvis_biphenyl.csv', mol_id=bp_id).data
# Insert UV-Vis data into database
session.add(UvVisData(**uvvis_data))
session.commit()

absorbance_rawdata = ProcessUvVis('databases_demo/raw_data/uvvis_biphenyl.csv', mol_id=bp_id).absorbance_data
# Convert each absorbance table row into a validated object
absorbance_data = [AbsorbanceData(**data) for data in absorbance_rawdata] 
# Insert Absorbance data into database
session.bulk_save_objects(absorbance_data) 
session.commit()

Here we loop through a dictionary of molecule names and their SMILES strings, and for each, we generate computational and experimental data and insert the data to the database tables.

In [ ]:
# Insert DFT and UV-Vis data for other molecules 

for name, mol_id in extra_mol_ids.items(): 
  # Extract and transform data
  gaussian_data = ProcessDFT('databases_demo/raw_data/tddft_'+name+'.log', mol_id=mol_id).data
  uvvis_data = ProcessUvVis('databases_demo/raw_data/uvvis_'+name+'.csv', mol_id=mol_id).data
  absorbance_rawdata = ProcessUvVis('databases_demo/raw_data/uvvis_'+name+'.csv', mol_id=mol_id).absorbance_data
  absorbance_data = [AbsorbanceData(**data) for data in absorbance_rawdata] 
  # Load data
  session.add(DftData(**gaussian_data))
  session.add(UvVisData(**uvvis_data))
  session.bulk_save_objects(absorbance_data) 
  # Commit insertions
  session.commit()

## 4. Query the database

### Basic Queries

Here we demonstrate basic database queries and basic data plotting using [pandas](https://pandas.pydata.org/) and [matplotlib](https://matplotlib.org/). A basic query contains two parts: selection and projection. The selection portion filters the data record(s) (rows for SQL) that will be returned. The projection specifies the record attribute(s) (columns for SQL) that will be shown. For example, imagine a researcher wants to know the SMILES strings for all molecules in a database that have a molecular weight more than 100 g/mol. The selection would stipulate only data records with a molecular weight greater than 100 g/mol, while the projection would specify the return of the SMILES attribute. Alternatively, the researcher might like to list the lowest-lying excited state energy for every molecule or find and count all molecules with more than ten atoms. Basic queries like this are quick and easy in both SQL and No-SQL databases, even when tens of thousands of molecules are present. 


<img src='https://raw.githubusercontent.com/D3TaLES/databases_demo/main/notebooks/media/queries.png' width="700" height="500">

In [ ]:
# View Molecules data table
pd.read_sql("molecules", engine.connect())

In [ ]:
# View UV-Vis data table
pd.read_sql("uvvis_data", engine.connect())

In [ ]:
# View DFT data table
pd.read_sql("dft_data", engine.connect())

In [ ]:
# Count the number of molecules in the database
pd.read_sql("SELECT COUNT(*) FROM molecules;", engine.connect())

In [ ]:
# Get molecules with more than 10 atoms
pd.read_sql("SELECT * FROM molecules WHERE number_of_atoms > 10;", engine.connect())

In [ ]:
# Get molecules with more than 10 atoms, showing only molecule IDs
pd.read_sql("SELECT * FROM molecules AS mols WHERE number_of_atoms > 10;", engine.connect())

In [ ]:
# Get all the SMILES string in the molecules database where the molecular weight is greater than 100 
pd.read_sql("SELECT smiles FROM molecules WHERE molecular_weight > 100;", engine.connect())

In [ ]:
# Search for all singlet excitation energy values in the database
pd.read_sql("SELECT mols.mol_id, mols.smiles, dft.first_excitation FROM dft_data AS dft INNER JOIN molecules AS mols ON (dft.mol_id=mols.mol_id);", engine.connect())

# Technically this query could work too (though it wouldn't give smiles), but it wouldn't demonstrate how to perform a table join 
# pd.read_sql("SELECT mol_id, first_excitation FROM dft_data;", engine.connect())

### Plotting
Here we demonstrate the data analysis examples from the paper: (1) Comparing computationally-estimated singlet excitation and experimentally-measured optical gap and (2) plotting spectrum only when the singlet excitation energy is greater than 4 eV.

In [ ]:
# Get the absorption spectrum data for cyclohexen-eylbenzene
query = pd.read_sql("SELECT wavelength, absorbance FROM absorbance_data WHERE mol_id LIKE 'cyclohexen-1-ylbenzene';", engine.connect())
# Plot data
query.plot(x='wavelength', y='absorbance')

#### EXAMPLE 1: Comparing computationally-estimated singlet excitation and experimentally-measured optical gap

<img src='https://raw.githubusercontent.com/D3TaLES/databases_demo/main/notebooks/media/ex1.png' width="400" height="550">




In [ ]:
# Gather data
query = pd.read_sql(""" 
  SELECT uvv.mol_id, uvv.optical_gap, dft.first_excitation 
  FROM dft_data AS dft INNER JOIN uvvis_data AS uvv 
  ON (dft.mol_id=uvv.mol_id);
""", engine.connect())
query

In [ ]:
# Plot data
fig, ax = plt.subplots(figsize=(4,3))

for i, mol in query.iterrows(): 
  plt.scatter(mol.optical_gap, mol.first_excitation, label=mol.mol_id)

# Add plot details 
plt.legend()
plt.xlabel('Optical Gap (eV)')
plt.ylabel('Singlet Excitation Energy (eV)')
plt.tight_layout()
plt.savefig('plot1.png', dpi=300)

#### EXAMPLE 2: Plotting spectrum only when the singlet excitation energy is greater than 4 eV

<img src='https://raw.githubusercontent.com/D3TaLES/databases_demo/main/notebooks/media/ex2.png' width="400" height="550">

In [ ]:
# Search for all singlet excitation values in the database
pd.read_sql("SELECT first_excitation FROM dft_data;", engine.connect())

In [ ]:
# Get the molecules wtih a single excitation greater than 4
molecules = pd.read_sql("SELECT mol_id from dft_data WHERE first_excitation > 4;", engine.connect())

# Plot absorption spectra for the molecules queried 
fig, ax = plt.subplots(figsize=(4.2,3))
for mol in molecules.mol_id: 
  query = pd.read_sql(f"SELECT wavelength, absorbance FROM absorbance_data WHERE mol_id LIKE '{mol}';", engine.connect())
  ax.plot(query.wavelength, query.absorbance, label=mol)

# Add details 
plt.legend()
plt.xlabel('Wavelength (nm)')
plt.ylabel('Absorption')
plt.tight_layout()
plt.savefig('plot2.png', dpi=300)

# !!! Reset Database !!!
Warning! Running the following cell will delete the databases you have constructred with the previous cells. To continue exploring this code after running this cell, you must rerun all cells starting with the `1. Initialize Database` step. 

In [ ]:
# Clean all tables! 
MetaData(bind=engine).drop_all(tables=[
                                         Molecules.__table__,
                                         DftData.__table__,
                                         Synonyms.__table__, 
                                         UvVisData.__table__,
                                         AbsorbanceData.__table__
                                         ])